In [1]:
import torch
import numpy as np
from skimage.measure import marching_cubes
from pytorch3d.structures import Meshes
from pytorch3d.io import save_obj
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader
)



In [2]:
from pysdf import SDF

# Load some mesh (don't necessarily need trimesh)
import trimesh
o = trimesh.load('data/cow_mesh/cow.obj')
f = SDF(o.vertices, o.faces); # (num_vertices, 3) and (num_faces, 3)

# Compute some SDF values (negative outside);
# takes a (num_points, 3) array, converts automatically
origin_sdf = f([0, 0, 0])
sdf_multi_point = f([[0, 0, 0],[1,1,1],[0.1,0.2,0.2]])

# Contains check
origin_contained = f.contains([0, 0, 0])

# Misc: nearest neighbor
origin_nn = f.nn([0, 0, 0])

# Misc: uniform surface point sampling
random_surface_points = f.sample_surface(10000)

# Misc: surface area
the_surface_area = f.surface_area

In [3]:
random_surface_points.shape

(10000, 3)

In [4]:

# mosaic_sdf = optimizer.model
resolution = 8
device = 'cuda'


# # Assuming 'mosaic_sdf' is your MosaicSDF instance and 'resolution' is the desired grid resolution
grid_points = torch.stack(torch.meshgrid(
    torch.linspace(-1, 1, resolution),
    torch.linspace(-1, 1, resolution),
    torch.linspace(-1, 1, resolution)
), dim=-1).reshape(-1, 3)#.to(device)

sdf_values = f(grid_points)
sdf_volume = sdf_values.reshape(resolution, resolution, resolution)

# # Get the SDF values at these points
# sdf_values = mosaic_sdf(grid_points).detach().cpu().numpy()
# sdf_volume = sdf_values.reshape(resolution, resolution, resolution)
# sdf_volume.shape

/home/che/miniforge3/envs/gen3d/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1706712676143/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:

# Run marching cubes to get vertices, faces, and normals
verts, faces, normals, _ = marching_cubes(sdf_volume, level=0)
faces = faces + 1  # skimage has 0-indexed faces, while PyTorch3D expects 1-indexed

# Convert to PyTorch tensors
verts = torch.tensor(verts.copy(), dtype=torch.float32)
faces = torch.tensor(faces.copy(), dtype=torch.int64)

# Create a PyTorch3D mesh
mesh = Meshes(verts=[verts], faces=[faces])

# Initialize a renderer
R, T = look_at_view_transform(2.7, 0, 90)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
raster_settings = RasterizationSettings(image_size=512)
lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=SoftPhongShader(device=device, cameras=cameras, lights=lights)
)

# Render the mesh
images = renderer(mesh.to(device))


In [20]:
verts.min(axis=0)

array([4.3094845, 2.061425 , 2.5206199], dtype=float32)